<a href="https://colab.research.google.com/github/RRM2021/USAID-MODELS/blob/main/IE_RIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESTRATEGIA DE INTEGRACIÓN REGIONAL RIS - USAID
# INICIATIVAS INTEGRADORAS REGIONALES
# RICARDO RAMÍREZ - ASESOR RIS

**OBJETIVO: Describir en forma cuantitativa y cualitativa las iniciativas integradoras regionales RIS.**

PREGUNTAS:

- ¿Cuántas y cuáles son las iniciativas integradoras de RIS en cada región?
- ¿Cuáles y cuántas iniciativas apoyan iniciativas PDET?
- ¿A cuántas y a cuáles iniciativas PDET apoyan las iniciativas integradoras RIS?
- ¿Cuántas y cuáles iniciativas están asociadas al capítulo étnico del Acuerdo de Paz?
- Cuántas y cuáles iniciativas integradoras RIS están alineadas con los planes de desarrollo departamentales?
- ¿En cuántos y cuáles municipios de cada región están las iniciativas integradoras de RIS?
- ¿Cuáles y cuántas Actividades de USAID participan en iniciativas integradoras de RIS?
- ¿Cuáles y cuántos tipos de aliados tienen las iniciativas integradoras RIS?
- ¿Cuáles y cuántos son los tipos de población objetivo de las iniciativas integradoras RIS?

**SE INSTALA PYSPARK PARA PROCESAR LOS DATOS CON ESTA HERRAMIENTA DE PROCESAMIENTO DE BIG DATA**

In [ ]:
try:
    import pyspark
    print("PySpark is already installed.")
except ImportError:
    print("PySpark is not installed. Installing now...")
    !pip install pyspark
    print("PySpark installation complete.")


PySpark is not installed. Installing now...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=e72647a9b1f4efa6fbf9f1c212604d0e5c7101e77ae08e110f129a9149bc9962
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
PySpark installation complete.


In [ ]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [ ]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

**SE IMPORTAN LAS HERRAMIENTAS PARA DESCARGAR DATOS DESDE GOOGLE DRIVE**

In [ ]:
import gspread
from google.colab import auth

In [ ]:
!pip install --upgrade google-api-python-client oauth2client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 80.6 MB/s eta 0:00:00
  Attempting uninstall: google-auth-httplib2
    Found existing installation: google-auth-httplib2 0.1.1
    Uninstalling google-auth-httplib2-0.1.1:
      Successfully uninstalled google-auth-httplib2-0.1.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0


In [ ]:
auth.authenticate_user()

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

**SE DESCARGAN LOS DATOS DESDE GOOGLE DRIVE, Y SE CREAN DATA FRAMES DE PYSPARK**

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('Iniciativas_RIS')

# Retrieve all values from the worksheet
data = worksheet.get_all_values()

In [ ]:
# Extract column titles from the first row
column_titles = data[0]

# Extract data rows excluding the first row
data_rows = data[1:]

# Create DataFrame with column titles
data_frame = spark.createDataFrame(data_rows, column_titles)

In [ ]:
data_frame.createOrReplaceTempView("IE_RIS")

# Show DataFrame
data_frame.show()

+---------+--------+-------------+-------------+--------------------+--------------------+---------------+--------------------+-------------------+------------+---------------------+
|ID_REGION|  ID_OIR|  ID_OBJETIVO|ID_INICIATIVA|        NOMBRE_CORTO|        NOMBRE_LARGO|INICIATIVA_PDET|    CODIGO_INIC_PDET|ASOCIADA_CAP_ETNICO|ASOCIADA_PDD|LINEA_ESTRATEGICA_PDD|
+---------+--------+-------------+-------------+--------------------+--------------------+---------------+--------------------+-------------------+------------+---------------------+
|      PCF|PCF_OIR1|PCF_OIR1_OBJ2|      PCF_IN1|Fortalecimiento d...|Fortalecimiento d...|             SÍ|1052473297579, 10...|                   |          SÍ| PROGRAMA 9. Parti...|
|      PCF|PCF_OIR1|PCF_OIR1_OBJ1|      PCF_IN2|Fortalecer las ca...|Implementar una e...|             SÍ|1052390270120, 10...|                   |          SÍ| PROGRAMA 2. Forta...|
|      PCF|PCF_OIR1|PCF_OIR1_OBJ2|      PCF_IN3|Proyecto de preve...|Implementa una a

In [ ]:
import pandas as pd
from google.colab import files

# Convert PySpark DataFrame to Pandas DataFrame
df = data_frame.toPandas()

# **INICIATIVAS INTEGRADORAS POR REGION**

In [ ]:
# Count unique values in 'SubCategory' and group by 'Category'
grouped_df = df.groupby('ID_REGION')['ID_INICIATIVA'].nunique().reset_index()
grouped_df.columns = ['ID_REGION', 'UniqueID_INICIATIVAS']

print(grouped_df)


  ID_REGION  UniqueID_INICIATIVAS
0       BCN                     5
1       BOG                     6
2       CAT                     4
3       CQT                     9
4       PCF                     8
5       SDC                     2


In [ ]:
try:
    import plotly
    print("plotly openpyxl is already installed.")
except ImportError:
    print("plotly openpyxl is not installed. Installing now...")
    !pip install plotly openpyxl
    print("plotly openpyxl installation complete.")

plotly openpyxl is already installed.


In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
# Sort the DataFrame by 'UniqueID_INICIATIVAS' in descending order
grouped_df_sorted = grouped_df.sort_values(by='UniqueID_INICIATIVAS', ascending=False)


# Create the bar plot
fig = px.bar(grouped_df_sorted, x='ID_REGION', y='UniqueID_INICIATIVAS', title='INICIATIVAS INTEGRADORAS POR REGION')

# Customize the plot
fig.update_layout(
    xaxis_title='REGIÓN',
    yaxis_title='NÚMERO DE INICIATIVAS',
    title={
        'text': 'INICIATIVAS INTEGRADORAS POR REGION',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()


In [ ]:
#spark.sql("SELECT COUNT(*) FROM IE_RIS")
# Write and execute the SQL query
result = spark.sql("SELECT COUNT(DISTINCT ID_INICIATIVA) AS numero_iniciativas FROM IE_RIS")

# Show the result
result.show()

+------------------+
|numero_iniciativas|
+------------------+
|                34|
+------------------+



# **INICIATIVAS INTEGRADORAS QUE APOYAN INICIATIVAS PDET POR REGION**

In [ ]:
# Count categories in 'Category' and group by 'Group'
grouped_df1 = df.groupby(['ID_REGION', 'INICIATIVA_PDET']).size().reset_index(name='Counts')

print(grouped_df1)


  ID_REGION INICIATIVA_PDET  Counts
0       BCN              NO       1
1       BCN              SÍ       4
2       BOG              NO       6
3       CAT              NO       2
4       CAT              SÍ       2
5       CQT              NO       5
6       CQT              SÍ       4
7       PCF              SÍ       8
8       SDC              NO       1
9       SDC              SÍ       1


In [ ]:
# Create the bar plot with permanent labels
fig = px.bar(grouped_df1, x='ID_REGION', y='Counts', color='INICIATIVA_PDET', barmode='group', title='ALINEACIÓN INICIATIVAS INTEGRADORAS CON PDET')

# Customize the plot
fig.update_layout(
    xaxis_title='REGIÓN',
    yaxis_title='NÚMERO DE INICIATIVAS',
    title={
        'text': 'ALINEACIÓN INICIATIVAS INTEGRADORAS CON PDET',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')


# Show the plot
fig.show()


In [ ]:
# Write and execute the SQL query to count occurrences of each unique value
result = spark.sql("SELECT INICIATIVA_PDET, COUNT(*) AS count FROM IE_RIS GROUP BY INICIATIVA_PDET")

# Show the result
result.show()


+---------------+-----+
|INICIATIVA_PDET|count|
+---------------+-----+
|             SÍ|   19|
|             NO|   15|
+---------------+-----+



# **NÚMERO Y CÓDIGO DE INICIATIVAS PDET APOYADAS POR INICIATIVAS INTEGRADORAS**

In [ ]:
# SQL query to count the number of elements in the 'COD_INIC_PDET' column
query = """
SELECT
    ID_INICIATIVA,
    CODIGO_INIC_PDET,
    size(split(CODIGO_INIC_PDET, ',')) AS inic_count
FROM IE_RIS
"""

result = spark.sql(query)

# Show the result
result.show()

+-------------+--------------------+----------+
|ID_INICIATIVA|    CODIGO_INIC_PDET|inic_count|
+-------------+--------------------+----------+
|      PCF_IN1|1052473297579, 10...|         6|
|      PCF_IN2|1052390270120, 10...|         9|
|      PCF_IN3|1052612293066, 10...|         2|
|      PCF_IN4|105283525278512, ...|         2|
|      PCF_IN5|1052612292775, 10...|         4|
|      PCF_IN6|1052835278611, 10...|         2|
|      PCF_IN7|1052473297513, 10...|         2|
|      PCF_IN8|1052612292789, 10...|         4|
|      BCN_IN1|305040283545, 030...|        10|
|      BCN_IN2|0305107297005, 03...|         3|
|      BCN_IN3|                    |         1|
|      BCN_IN4|        305895297004|         1|
|      BCN_IN5|305120303371, 305...|         8|
|      SDC_IN1|                    |         1|
|      SDC_IN2|1423807174865, 14...|         5|
|      CAT_IN1|        454720199470|         1|
|      CAT_IN2|        454720202047|         1|
|      CAT_IN3|                    |    

In [ ]:
from pyspark.sql.functions import split, size

# Split the column and count the number of elements in each row
data_frame_split = data_frame.withColumn("inic_count", size(split(data_frame["CODIGO_INIC_PDET"], ",")))

# Sum the counts
total_inic_count = data_frame_split.agg({'inic_count': 'sum'}).collect()[0][0]
print(f"Total count of elements in 'CODIGO_INIC_PDET': {total_inic_count}")

Total count of elements in 'CODIGO_INIC_PDET': 79


79 Iniciativas PDET, con su respectivo código, son apoyadas por Iniciativas Integradoras de RIS

# **ALINEACIÓN DE INICIATIVAS PDET CON PLANES DE DESARROLLO DEPARTAMENTAL**

In [ ]:
# Write and execute the SQL query to count occurrences of each unique value
result = spark.sql("SELECT ASOCIADA_PDD, COUNT(*) AS count FROM IE_RIS GROUP BY ASOCIADA_PDD")

# Show the result
result.show()

+------------+-----+
|ASOCIADA_PDD|count|
+------------+-----+
|          SÍ|   34|
+------------+-----+



Todas las Iniciativas Integradoras de RIS están alineadas con los Planes de Desarrollo Departamentales, en sus regiones.

# **MUNICIPIOS EN INICIATIVAS INTEGRADORAS**

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('INIC_MPIOS')

# Retrieve all values from the worksheet
data_1 = worksheet.get_all_values()

# Extract column titles from the first row
column_titles_1 = data_1[0]

# Extract data rows excluding the first row
data_rows_1 = data_1[1:]

# Create DataFrame with column titles
data_frame_1 = spark.createDataFrame(data_rows_1, column_titles_1)

data_frame_1.createOrReplaceTempView("IE_MPIOS")

# Show DataFrame
data_frame_1.show()

+-------------+--------------------+
|ID_INICIATIVA|          MUNICIPIOS|
+-------------+--------------------+
|      PCF_IN1|           BARBACOAS|
|      PCF_IN1|           EL CHARCO|
|      PCF_IN1|         MAGÜI PAYAN|
|      PCF_IN1|            RICAURTE|
|      PCF_IN1|       ROBERTO PAYÁN|
|      PCF_IN1|SAN ANDRES DE TUMACO|
|      PCF_IN2|           BARBACOAS|
|      PCF_IN2|           EL CHARCO|
|      PCF_IN2|   FRANCISCO PIZARRO|
|      PCF_IN2|             LA TOLA|
|      PCF_IN2|            MOSQUERA|
|      PCF_IN2|         MAGÜI PAYAN|
|      PCF_IN2|       OLAYA HERRERA|
|      PCF_IN2|            RICAURTE|
|      PCF_IN2|       ROBERTO PAYÁN|
|      PCF_IN2|SAN ANDRES DE TUMACO|
|      PCF_IN2|       SANTA BÁRBARA|
|      PCF_IN3|           BARBACOAS|
|      PCF_IN3|           EL CHARCO|
|      PCF_IN3|         MAGÜI PAYAN|
+-------------+--------------------+
only showing top 20 rows



In [ ]:
data_frame_5 = spark.sql("SELECT ier.ID_REGION, iem.ID_INICIATIVA, ier.NOMBRE_CORTO, MUNICIPIOS FROM IE_RIS AS ier, IE_MPIOS AS iem WHERE ier.ID_INICIATIVA = iem.ID_INICIATIVA")
data_frame_5.show()


+---------+-------------+--------------------+----------+
|ID_REGION|ID_INICIATIVA|        NOMBRE_CORTO|MUNICIPIOS|
+---------+-------------+--------------------+----------+
|      BCN|      BCN_IN1|      Alianza Anorí |     ANORÍ|
|      BCN|      BCN_IN2|Educación Financi...|   BRICEÑO|
|      BCN|      BCN_IN2|Educación Financi...|   ITUANGO|
|      BCN|      BCN_IN2|Educación Financi...|  VALDIVIA|
|      BCN|      BCN_IN3|Lo que cuentan lo...|     NECHÍ|
|      BCN|      BCN_IN3|Lo que cuentan lo...|   CÁCERES|
|      BCN|      BCN_IN3|Lo que cuentan lo...|    TARAZÁ|
|      BCN|      BCN_IN3|Lo que cuentan lo...|  CAUCASIA|
|      BCN|      BCN_IN3|Lo que cuentan lo...|  EL BAGRE|
|      BCN|      BCN_IN3|Lo que cuentan lo...|  ZARAGOZA|
|      BCN|      BCN_IN4|Tejiendo Justicia...|  CAUCASIA|
|      BCN|      BCN_IN4|Tejiendo Justicia...|  EL BAGRE|
|      BCN|      BCN_IN4|Tejiendo Justicia...|    TARAZÁ|
|      BCN|      BCN_IN4|Tejiendo Justicia...|   CÁCERES|
|      BCN|   

In [ ]:
# Convert PySpark DataFrame to Pandas DataFrame
df_2 = data_frame_5.toPandas()

# Count unique values in 'SubCategory' and group by 'Category'
grouped_df_mpios = df_2.groupby('ID_REGION')['MUNICIPIOS'].nunique().reset_index()
grouped_df_mpios.columns = ['ID_REGION', 'Uniquemunicipios']

print(grouped_df_mpios)

  ID_REGION  Uniquemunicipios
0       BCN                10
1       BOG                 2
2       CAT                 8
3       CQT                16
4       PCF                11
5       SDC                 5


In [ ]:
# Sort the DataFrame by 'UniqueID_INICIATIVAS' in descending order
grouped_df_mpios_sorted = grouped_df_mpios.sort_values(by='Uniquemunicipios', ascending=False)


# Create the bar plot
fig = px.bar(grouped_df_mpios_sorted, x='ID_REGION', y='Uniquemunicipios', title='MUNICIPIOS EN INICIATIVAS POR REGION')

# Customize the plot
fig.update_layout(
    xaxis_title='REGIÓN',
    yaxis_title='NÚMERO DE MUNICIPIOS',
    title={
        'text': 'MUNICIPIOS EN INICIATIVAS INTEGRADORAS POR REGION',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()


In [ ]:
#spark.sql("SELECT COUNT(*) FROM IE_RIS")
# Write and execute the SQL query
result = spark.sql("SELECT COUNT(DISTINCT MUNICIPIOS) AS numero_municipios FROM IE_MPIOS")

# Show the result
result.show()

+-----------------+
|numero_municipios|
+-----------------+
|               52|
+-----------------+



Las 34 Iniciativas Integradoras están localizadas en 52 municipios de la geografía de USAID, en seis regiones de Colombia

# **ACTIVIDADES DE USAID EN INICIATIVAS INTEGRADORAS**

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('INIC_ACT_USAID')

# Retrieve all values from the worksheet
data_2 = worksheet.get_all_values()

# Extract column titles from the first row
column_titles_2 = data_2[0]

# Extract data rows excluding the first row
data_rows_2 = data_2[1:]

# Create DataFrame with column titles
data_frame_2 = spark.createDataFrame(data_rows_2, column_titles_2)

data_frame_2.createOrReplaceTempView("ACTIV")

# Show DataFrame
data_frame_2.show()

+-------------+--------------------+------------+
|ID_INICIATIVA|    NOMBRE_ACTIVIDAD|ID_ACTIVIDAD|
+-------------+--------------------+------------+
|      PCF_IN1|Programa Justicia...|          JI|
|      PCF_IN1|Colombia Transfor...|         OTI|
|      PCF_IN1|         Inspira Paz| INSPIRA PAZ|
|      PCF_IN1|Restaurando Nuest...|         ROF|
|      PCF_IN2|Generando Equidad...|         GEP|
|      PCF_IN2|Colombia Transfor...|         OTI|
|      PCF_IN2|Programa de Jóven...|         YRA|
|      PCF_IN2|Destino Naturalez...|          DN|
|      PCF_IN2|Programa Justicia...|          JI|
|      PCF_IN2|         Inspira Paz| INSPIRA PAZ|
|      PCF_IN2|Restaurando Nuest...|         ROF|
|      PCF_IN2|Somos Comunidad -...|          RC|
|      PCF_IN2|Gobernabilidad Re...|          RG|
|      PCF_IN3|Somos Comunidad -...|          RC|
|      PCF_IN3|Restaurando Nuest...|         ROF|
|      PCF_IN3|Generando Equidad...|         GEP|
|      PCF_IN4|Generando Equidad...|         GEP|


In [ ]:
data_frame_6 = spark.sql("SELECT ier.ID_REGION, act.ID_INICIATIVA, act.ID_ACTIVIDAD FROM IE_RIS AS ier, ACTIV AS act WHERE ier.ID_INICIATIVA = act.ID_INICIATIVA")
data_frame_6.show()

+---------+-------------+------------+
|ID_REGION|ID_INICIATIVA|ID_ACTIVIDAD|
+---------+-------------+------------+
|      BCN|      BCN_IN1|         TDA|
|      BCN|      BCN_IN1|         WLH|
|      BCN|      BCN_IN1|         SAA|
|      BCN|      BCN_IN1|         GEP|
|      BCN|      BCN_IN1|          FE|
|      BCN|      BCN_IN2|         GEP|
|      BCN|      BCN_IN2|          FE|
|      BCN|      BCN_IN2|         SAA|
|      BCN|      BCN_IN3|         JXT|
|      BCN|      BCN_IN3|          RG|
|      BCN|      BCN_IN3|         OTI|
|      BCN|      BCN_IN4|         OTI|
|      BCN|      BCN_IN4|          JI|
|      BCN|      BCN_IN4|         GEP|
|      BCN|      BCN_IN4|          RC|
|      BCN|      BCN_IN4|        HR4P|
|      BCN|      BCN_IN4|          RG|
|      BCN|      BCN_IN5|         OTI|
|      BCN|      BCN_IN5|         GEP|
|      BCN|      BCN_IN5|        HR4P|
+---------+-------------+------------+
only showing top 20 rows



In [ ]:
# Convert PySpark DataFrame to Pandas DataFrame
df_3 = data_frame_6.toPandas()

# Count unique values in 'SubCategory' and group by 'Category'
grouped_df_act = df_3.groupby('ID_REGION')['ID_ACTIVIDAD'].nunique().reset_index()
grouped_df_act.columns = ['ID_REGION', 'UniqueACTIVIDAD']

print(grouped_df_act)

  ID_REGION  UniqueACTIVIDAD
0       BCN               12
1       BOG                7
2       CAT                9
3       CQT               12
4       PCF               12
5       SDC                8


In [ ]:
# Sort the DataFrame by 'UniqueID_INICIATIVAS' in descending order
grouped_df_act_sorted = grouped_df_act.sort_values(by='UniqueACTIVIDAD', ascending=False)


# Create the bar plot
fig = px.bar(grouped_df_act_sorted, x='ID_REGION', y='UniqueACTIVIDAD', title='ACTIVIDADES DE USAID EN INICIATIVAS')

# Customize the plot
fig.update_layout(
    xaxis_title='REGIÓN',
    yaxis_title='NÚMERO DE ACTIVIDADES',
    title={
        'text': 'ACTIVIDADES DE USAID EN INICIATIVAS INTEGRADORAS POR REGION',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

In [ ]:
#spark.sql("SELECT COUNT(*) FROM IE_RIS")
# Write and execute the SQL query
result = spark.sql("SELECT COUNT(DISTINCT ID_ACTIVIDAD) AS numero_actividades FROM ACTIV")

# Show the result
result.show()

+------------------+
|numero_actividades|
+------------------+
|                28|
+------------------+



In [ ]:
# Create the treemap
fig = px.treemap(df_3, path=[px.Constant(""), 'ID_REGION', 'ID_ACTIVIDAD'],
                 hover_name='ID_REGION', color='ID_ACTIVIDAD', color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(root_color="lightgray")
fig.update_layout(margin=dict(t=40, l=25, r=25, b=25))
fig.update_layout(title='INICIATIVAS INTEGRADORAS - ACTIVIDADES USAID')
fig.update_layout(title_font_color='red')
fig.update_layout(title_font_family="Arial")
fig.update_layout(title_font_size=15)
fig.update_layout(width=1100, height=600)
fig.update_traces(maxdepth=6, selector=dict(type='treemap'))
fig.update_traces(hoverinfo="current path", selector=dict(type='treemap'))
# Enlarge text size for all labels in the treemap
fig.update_traces(textfont=dict(size=25))
# Display the plot
fig.show()

# **ALIADOS EN INICIATIVAS INTEGRADORAS**

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('INIC_ALIADOS')

# Retrieve all values from the worksheet
data_3 = worksheet.get_all_values()

# Extract column titles from the first row
column_titles_3 = data_3[0]

# Extract data rows excluding the first row
data_rows_3 = data_3[1:]

# Create DataFrame with column titles
data_frame_3 = spark.createDataFrame(data_rows_3, column_titles_3)

data_frame_3.createOrReplaceTempView("ALIADOS")

# Show DataFrame
data_frame_3.show()

+-------------+--------------------+--------------+--------------------+
|ID_INICIATIVA|        TIPO_ALIADOS|NUMERO_ALIADOS|      NOMBRE_ALIADOS|
+-------------+--------------------+--------------+--------------------+
|      BCN_IN1|   Gobierno nacional|             0|Agencia de Renova...|
|      BCN_IN1|   Gobiernos Locales|             1|      Alcaldía Anorí|
|      BCN_IN1|     Empresa privada|             3|COMFAMA, AGROSAVI...|
|      BCN_IN1|     Empresa privada|             1|Corporación Gilbe...|
|      BCN_IN1|Gobierno departam...|             1|Gobernación de An...|
|      BCN_IN1|   Gobierno nacional|             0|                SENA|
|      BCN_IN1|            Academia|             1|Universidad de An...|
|      BCN_IN2|   Gobiernos Locales|             3|Alcaldías Valdivi...|
|      BCN_IN2|   Gobierno nacional|             1|      Banco Agrario |
|      BCN_IN2|     Empresa privada|             1|Cooperativa Finan...|
|      BCN_IN3|   Gobiernos Locales|             6|

In [ ]:
# Write and execute the SQL query to count occurrences of each unique value
result = spark.sql("SELECT TIPO_ALIADOS, COUNT(*) AS count FROM ALIADOS GROUP BY TIPO_ALIADOS")

# Show the result
result.show()


+--------------------+-----+
|        TIPO_ALIADOS|count|
+--------------------+-----+
|   Gobierno nacional|   27|
|             Otro/as|    4|
|Autoridades Étnic...|   12|
|     Empresa privada|   19|
|Esquema asociativ...|    4|
|Organizaciones de...|    6|
|Cooperación inter...|    8|
|Organizaciones so...|   15|
|            Academia|   14|
|Espacios de parti...|    5|
|   Gobiernos Locales|   41|
|Gobierno departam...|   13|
|  Ministerio Público|    6|
+--------------------+-----+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum

# Group by 'category' and sum the 'value' column
grouped_sum_df = data_frame_3.groupBy("iD_INICIATIVA").agg(spark_sum("NUMERO_ALIADOS").alias("sum_aliados"))

# Show the result
grouped_sum_df.show()

+-------------+-----------+
|iD_INICIATIVA|sum_aliados|
+-------------+-----------+
|      BCN_IN3|       10.0|
|      CQT_IN4|        1.0|
|      PCF_IN1|       27.0|
|      CQT_IN5|        5.0|
|      CQT_IN3|        6.0|
|      CQT_IN7|        1.0|
|      BOG_IN4|        0.0|
|      CQT_IN1|        4.0|
|      BCN_IN5|        6.0|
|      CQT_IN9|        7.0|
|      BCN_IN4|        4.0|
|      CAT_IN4|       11.0|
|      CAT_IN1|        6.0|
|      BCN_IN2|        5.0|
|      CQT_IN6|       32.0|
|      BCN_IN1|        7.0|
|      CQT_IN8|        1.0|
|      BOG_IN2|        6.0|
|      CAT_IN3|        5.0|
|      CQT_IN2|        2.0|
+-------------+-----------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum

# Group by 'category' and sum the 'value' column
grouped_sum_df4 = data_frame_3.groupBy("TIPO_ALIADOS").agg(spark_sum("NUMERO_ALIADOS").alias("sum_tipo_aliados"))

# Show the result
grouped_sum_df4.show()

+--------------------+----------------+
|        TIPO_ALIADOS|sum_tipo_aliados|
+--------------------+----------------+
|   Gobierno nacional|            19.0|
|             Otro/as|             3.0|
|Autoridades Étnic...|            11.0|
|     Empresa privada|            33.0|
|Esquema asociativ...|             6.0|
|Organizaciones de...|            14.0|
|Cooperación inter...|             7.0|
|Organizaciones so...|            24.0|
|            Academia|            11.0|
|Espacios de parti...|            21.0|
|   Gobiernos Locales|           104.0|
|Gobierno departam...|            10.0|
|  Ministerio Público|            28.0|
+--------------------+----------------+



In [ ]:
import pandas as pd
from google.colab import files

# Convert PySpark DataFrame to Pandas DataFrame
grouped_sum_pd_df4 = grouped_sum_df4.toPandas()

# Sort the DataFrame by 'UniqueID_INICIATIVAS' in descending order
grouped_sum_pd_df4_sorted = grouped_sum_pd_df4.sort_values(by='sum_tipo_aliados', ascending=False)


# Create the bar plot
fig = px.bar(grouped_sum_pd_df4_sorted, x='TIPO_ALIADOS', y='sum_tipo_aliados', title='TIPO DE ALIADOS EN INICIATIVAS')

# Customize the plot
fig.update_layout(
    xaxis_title='TIPO DE ALIADO',
    yaxis_title='NÚMERO',
    title={
        'text': 'TIPO DE ALIADOS EN INICIATIVAS INTEGRADORAS',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

# **POBLACIÓN OBJETIVO EN INICIATIVAS INTEGRADORAS**

In [ ]:
spreadsheet = gc.open('Tablas de Base de Datos RIS_1_07_2024')
worksheet = spreadsheet.worksheet('INIC_POBL_OBJ')

# Retrieve all values from the worksheet
data_4 = worksheet.get_all_values()

# Extract column titles from the first row
column_titles_4 = data_4[0]

# Extract data rows excluding the first row
data_rows_4 = data_4[1:]

# Create DataFrame with column titles
data_frame_4 = spark.createDataFrame(data_rows_4, column_titles_4)

data_frame_4.createOrReplaceTempView("POB_OBJ")

# Show DataFrame
data_frame_4.show()

+-------------+--------------------+-------------+--------------------+
|ID_INICIATIVA|      TIPO_POBLACION|NUM_POBLACION|    NOMBRE_POBLACION|
+-------------+--------------------+-------------+--------------------+
|      BCN_IN1|Funcionario/as pú...|             |   Alcaldía de Anorí|
|      BCN_IN1|             Jóvenes|             |Jóvenes Instituci...|
|      BCN_IN1|        Campesino/as|             |Juntas de acción ...|
|      BCN_IN1|             Mujeres|             |Mujeres vinculada...|
|      BCN_IN1|Organizaciones de...|             |Organizaciones de...|
|      BCN_IN2|        Campesino/as|          376|Familias caficult...|
|      BCN_IN2|        Campesino/as|           77|Familias particip...|
|      BCN_IN2|             Mujeres|          200|mujeres participa...|
|      BCN_IN3|             General|             |  Comunidad general |
|      BCN_IN3| Lideres/as sociales|           60|Redes de Consejer...|
|      BCN_IN4|           Indígenas|           40|Las comunidade

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum

# Group by 'category' and sum the 'value' column
grouped_sum_df2 = data_frame_4.groupBy("iD_INICIATIVA").agg(spark_sum("NUM_POBLACION").alias("sum_poblacion"))

# Show the result
grouped_sum_df2.show()


+-------------+-------------+
|iD_INICIATIVA|sum_poblacion|
+-------------+-------------+
|      BCN_IN3|         60.0|
|      CQT_IN4|         18.0|
|      CQT_IN5|        240.0|
|      CQT_IN3|         71.0|
|      CQT_IN1|         83.0|
|      BCN_IN5|        700.0|
|      BCN_IN4|         40.0|
|      CAT_IN4|          9.0|
|      CAT_IN1|       1369.0|
|      BCN_IN2|        653.0|
|      BCN_IN1|         NULL|
|      BOG_IN2|         NULL|
|      CAT_IN3|          8.0|
|      CQT_IN2|         40.0|
|      CAT_IN2|         25.0|
|      SDC_IN2|         NULL|
|      PCF_IN1|       1296.0|
|      PCF_IN7|       1430.0|
|      PCF_IN5|         45.0|
|      CQT_IN7|         10.0|
+-------------+-------------+
only showing top 20 rows



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as spark_sum

# Group by 'category' and sum the 'value' column
grouped_sum_df3 = data_frame_4.groupBy("TIPO_POBLACION").agg(spark_sum("NUM_POBLACION").alias("sum_tipo_pobl"))

# Show the result
grouped_sum_df3.show()

+--------------------+-------------+
|      TIPO_POBLACION|sum_tipo_pobl|
+--------------------+-------------+
|           Indígenas|        669.0|
|Víctimas del conf...|         NULL|
|             Otro/as|         51.0|
| Lideres/as sociales|        460.0|
|             General|         90.0|
|Esquema asociativ...|          5.0|
|             Mujeres|       1040.0|
|Organizaciones de...|         10.0|
|Organizaciones so...|         33.0|
|        Campesino/as|       5044.0|
|Funcionario/as pú...|        418.0|
|             Jóvenes|       2264.0|
|           Migrantes|         NULL|
|   Gobiernos Locales|         35.0|
|Unidades Productivas|         30.0|
|   Afrocolombiano/as|       1000.0|
+--------------------+-------------+



In [ ]:
import pandas as pd
from google.colab import files

# Convert PySpark DataFrame to Pandas DataFrame
grouped_sum_pd_df3 = grouped_sum_df3.toPandas()

In [ ]:
# Sort the DataFrame by 'UniqueID_INICIATIVAS' in descending order
grouped_sum_pd_df3_sorted = grouped_sum_pd_df3.sort_values(by='sum_tipo_pobl', ascending=False)


# Create the bar plot
fig = px.bar(grouped_sum_pd_df3_sorted, x='TIPO_POBLACION', y='sum_tipo_pobl', title='TIPO DE POBLACIÓN EN INICIATIVAS')

# Customize the plot
fig.update_layout(
    xaxis_title='TIPO DE POBLACIÓN',
    yaxis_title='NÚMERO',
    title={
        'text': 'TIPO DE POBLACIÓN EN INICIATIVAS INTEGRADORAS',
        'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    font=dict(
        family="Arial",
        size=15,
        color="black"
    )
)

# Update the text position to be inside the bars
fig.update_traces(texttemplate='%{y}', textposition='inside')

# Show the plot
fig.show()

In [ ]:
# List of columns to count unique values for
columns_to_count = ["ID_REGION", "INICIATIVA_PDET", "ASOCIADA_CAP_ETNICO", "ASOCIADA_PDD"  ]

# Loop through each column and execute the SQL query
for col in columns_to_count:
    query = f"SELECT {col}, COUNT(*) AS count FROM IE_RIS GROUP BY {col}"
    result = spark.sql(query)
    print(f"Unique values and their counts for column '{col}':")
    result.show()

Unique values and their counts for column 'ID_REGION':
+---------+-----+
|ID_REGION|count|
+---------+-----+
|      SDC|    2|
|      BCN|    5|
|      PCF|    8|
|      CAT|    4|
|      CQT|    9|
|      BOG|    6|
+---------+-----+

Unique values and their counts for column 'INICIATIVA_PDET':
+---------------+-----+
|INICIATIVA_PDET|count|
+---------------+-----+
|             SÍ|   19|
|             NO|   15|
+---------------+-----+

Unique values and their counts for column 'ASOCIADA_CAP_ETNICO':
+-------------------+-----+
|ASOCIADA_CAP_ETNICO|count|
+-------------------+-----+
|                   |   21|
|                 NO|   13|
+-------------------+-----+

Unique values and their counts for column 'ASOCIADA_PDD':
+------------+-----+
|ASOCIADA_PDD|count|
+------------+-----+
|          SÍ|   34|
+------------+-----+

